# Evaluation of Token Classification with OpenAI GPT Models


## Import Python packages

In [1]:
from os import listdir
from os.path import isfile, join
import json

import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

from datasets import load_dataset
from utils import filter_ner_io

## Import dataset

In this experiment, we use the GeoEDdA dataset which contains semantic annotations (at the token and span levels) for named entities (i.e., Spatial, Person, and Misc), nominal entities, spatial relations, and geographic coordinates. Nested named entities also present in this dataset were not considered in this experiment.

The dataset is available in the HuggingFace Hub: https://huggingface.co/datasets/GEODE/GeoEDdA

* Load the GeoEDdA dataset from the HuggingFace Hub:

In [2]:
dataset = load_dataset("GEODE/GeoEDdA")
test_set = pd.DataFrame(dataset['test'])
test_set.head()

,text,meta,tokens,spans
0,"COMPIEGNE, (Géog. mod.) ville de France, dans ...","{'volume': 3, 'head': 'COMPIEGNE', 'author': '...","[{'text': 'COMPIEGNE', 'start': 0, 'end': 9, '...","[{'text': 'COMPIEGNE', 'start': 0, 'end': 9, '..."
1,"HILPERHAUSEN, (Géog.) ville d'Allemagne en Fra...","{'volume': 8, 'head': 'HILPERHAUSEN', 'author'...","[{'text': 'HILPERHAUSEN', 'start': 0, 'end': 1...","[{'text': 'HILPERHAUSEN', 'start': 0, 'end': 1..."
2,"Patane ou Patany, (Géog. mod.) royaume des Ind...","{'volume': 12, 'head': 'Patane ou Patany', 'au...","[{'text': 'Patane', 'start': 0, 'end': 6, 'id'...","[{'text': 'Patane ou Patany', 'start': 0, 'end..."
3,"* ABYDE ou ABYDOS, sub. Ville maritime de Phry...","{'volume': 1, 'head': 'ABYDE ou ABYDOS', 'auth...","[{'text': '*', 'start': 0, 'end': 1, 'id': 0, ...","[{'text': 'ABYDE ou ABYDOS', 'start': 2, 'end'..."
4,"DUSSELDORP, (Géog. mod.) ville du cercle de We...","{'volume': 5, 'head': 'DUSSELDORP', 'author': ...","[{'text': 'DUSSELDORP', 'start': 0, 'end': 10,...","[{'text': 'DUSSELDORP', 'start': 0, 'end': 10,..."


* Add a new column with the list of tags (one tag per token):

In [3]:
tagset = ['Domain-mark','Head','NC-Person','NC-Spatial','NP-Misc','NP-Person','NP-Spatial','Relation','Latlong', 'ENE-Spatial', 'ENE-Person', 'ENE-Misc']

test_set['tags'] = test_set.apply(lambda x: filter_ner_io(x, tagset), axis=1)
test_set.head()

,text,meta,tokens,spans,tags
0,"COMPIEGNE, (Géog. mod.) ville de France, dans ...","{'volume': 3, 'head': 'COMPIEGNE', 'author': '...","[{'text': 'COMPIEGNE', 'start': 0, 'end': 9, '...","[{'text': 'COMPIEGNE', 'start': 0, 'end': 9, '...","[[Head], [O], [O], [Domain-mark], [Domain-mark..."
1,"HILPERHAUSEN, (Géog.) ville d'Allemagne en Fra...","{'volume': 8, 'head': 'HILPERHAUSEN', 'author'...","[{'text': 'HILPERHAUSEN', 'start': 0, 'end': 1...","[{'text': 'HILPERHAUSEN', 'start': 0, 'end': 1...","[[Head], [O], [O], [Domain-mark], [Domain-mark..."
2,"Patane ou Patany, (Géog. mod.) royaume des Ind...","{'volume': 12, 'head': 'Patane ou Patany', 'au...","[{'text': 'Patane', 'start': 0, 'end': 6, 'id'...","[{'text': 'Patane ou Patany', 'start': 0, 'end...","[[Head], [Head], [Head], [O], [O], [Domain-mar..."
3,"* ABYDE ou ABYDOS, sub. Ville maritime de Phry...","{'volume': 1, 'head': 'ABYDE ou ABYDOS', 'auth...","[{'text': '*', 'start': 0, 'end': 1, 'id': 0, ...","[{'text': 'ABYDE ou ABYDOS', 'start': 2, 'end'...","[[O], [Head], [Head], [Head], [O], [O], [O], [..."
4,"DUSSELDORP, (Géog. mod.) ville du cercle de We...","{'volume': 5, 'head': 'DUSSELDORP', 'author': ...","[{'text': 'DUSSELDORP', 'start': 0, 'end': 10,...","[{'text': 'DUSSELDORP', 'start': 0, 'end': 10,...","[[Head], [O], [O], [Domain-mark], [Domain-mark..."


## Evaluation

Several iterations of the token classification task have been performed. The predictions from all the iterations can be loaded and evaluated.

In [25]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

def load_predictions(path):
    predictions = []
    for run in sorted(listdir(path)):
        files = sorted([f for f in listdir(join(path, run)) if isfile(join(path, run, f))])
        preds = []
        #print(files)
        for file in files:
            with open(join(path,run, file), encoding='utf-8') as f:
                preds.append(json.load(f))
        predictions.append(preds)

    return predictions

def formatting_ner(pred_sentence, true_sentence):
    formatted_pred_sentence = []
    i = 0
    j = 0

    if pred_sentence is not None:
        pred_entities = pred_sentence.get('entities', [])
        while i < len(pred_entities) and j < len(true_sentence['tokens']):
            pred_token = pred_entities[i]
            true_token = true_sentence['tokens'][j]

            if 'text' not in pred_token or pred_token['text'] != true_token['text']:
                formatted_pred_sentence.append(['O'])
                j += 1
                continue

            labels = pred_token.get('labels', pred_token.get('label', ['O']))
            if isinstance(labels, str):
                labels = [labels]
            formatted_pred_sentence.append(sorted(labels))
            i += 1
            j += 1

    # Handle remaining true tokens
    while j < len(true_sentence['tokens']):
        formatted_pred_sentence.append(['O'])
        j += 1

    return formatted_pred_sentence

#def formatting_ner(pred_sentence, true_sentence):
#    formatted_pred_sentence = []
#    i = 0
#    j = 0
#    if pred_sentence is not None:
#        while(i < len(pred_sentence['entities']) and j < len(true_sentence['tokens'])):
#            if('text' not in list(pred_sentence['entities'][i].keys())):
#                formatted_pred_sentence.append('O')
#                i += 1
#                j += 1
#            elif(pred_sentence['entities'][i]['text'] == true_sentence['tokens'][j]['text'] and 'labels' in list(pred_sentence['entities'][i].keys())):
#                formatted_pred_sentence.append(pred_sentence['entities'][i]['labels'])
#                i += 1
#                j += 1
#            else:
#                j += 1
#                formatted_pred_sentence.append('O')
#    while(j < len(true_sentence['tokens'])):
#        formatted_pred_sentence.append('O')
#        j += 1
#    return formatted_pred_sentence


def format_sentences(pred, true):
    formatted_pred_sentences = []
    for pred_sentence, true_sentence in zip(pred, true.iterrows()):
        formatted_pred_sentences.append(formatting_ner(pred_sentence,true_sentence[1]))
    return formatted_pred_sentences


def flatten_multilabel(true_tags_column):
    """
    Flattens a list of list of tags (sentence → tokens → tag(s)) into
    a flat list of sets per token, handling both list-of-labels and single-label formats.
    
    Case 1: tag is a list of labels → {'NP-Spatial', 'ENE-Spatial'}
    Case 2: tag is a single label → {'NP-Spatial'}
    """
    all_tags = []
    for sentence in true_tags_column:
        for token_tag in sentence:
            if isinstance(token_tag, list):
                # Case 1: Already a list of labels
                all_tags.append(set(token_tag))
            elif isinstance(token_tag, str):
                # Case 2: Single label
                all_tags.append(set([token_tag]))
            else:
                # Fallback
                all_tags.append(set())
    return all_tags

def format_predictions_multilabel(predictions, test_set):
    """
    Wraps predicted single-labels in sets, treating 'O' as empty set.
    """
    formatted_pred_sentences = format_sentences(predictions, test_set)
    return [
        set([tag]) if isinstance(tag, str) and tag != 'O' else set()
        for sentence in formatted_pred_sentences for tag in sentence
    ]



def evaluate_multilabel(predictions, test_set, tagset):
    """
    Évalue un modèle NER multilabel en comparant les prédictions avec les vraies étiquettes.
    Prend en charge les cas où les étiquettes sont soit des chaînes uniques soit des listes de labels.
    """

    all_true_labels = []
    all_pred_labels = []

    for pred_sentence, (_, true_sentence) in zip(predictions, test_set.iterrows()):
        pred_formatted = formatting_ner(pred_sentence, true_sentence)
        true_formatted = true_sentence['tags']

        # Assurer cohérence : transformer chaque tag en set
        pred_sets = [set(tags) if isinstance(tags, list) else set([tags]) for tags in pred_formatted]
        true_sets = [set(tags) if isinstance(tags, list) else set([tags]) for tags in true_formatted]

        all_pred_labels.extend(pred_sets)
        all_true_labels.extend(true_sets)

    # Binarisation multilabel
    mlb = MultiLabelBinarizer(classes=sorted(set(tagset)))
    y_true = mlb.fit_transform(all_true_labels)
    y_pred = mlb.transform(all_pred_labels)

    # Calcul des scores
    precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None, zero_division=0)
    macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
    micro_precision, micro_recall, micro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='micro', zero_division=0)

    # Organisation des scores
    metrics = {
        tag: {
            'precision': p,
            'recall': r,
            'f1-score': f,
            'support': s
        } for tag, p, r, f, s in zip(mlb.classes_, precision, recall, f1, support)
    }

    # Ajout des moyennes globales
    metrics['macro avg'] = {
        'precision': macro_precision,
        'recall': macro_recall,
        'f1-score': macro_f1,
        'support': sum(support)
    }
    metrics['micro avg'] = {
        'precision': micro_precision,
        'recall': micro_recall,
        'f1-score': micro_f1,
        'support': sum(support)
    }

    return metrics



def evaluate(predictions, test_set, tagset):
    classification_reports = []  
    for prediction in predictions:
        classification_reports.append(evaluate_multilabel(prediction, test_set, tagset))
    return classification_reports


def get_avg_scores(classification_reports, score='f1-score'):
    avg_scores = {}
    for report in classification_reports:
        for tag in report.keys():
            if tag not in avg_scores:
                avg_scores[tag] = []
            avg_scores[tag].append(report[tag][score])

    avg_scores = {tag: sum(scores)/len(scores) for tag, scores in avg_scores.items()}
    return avg_scores


def bar_plot(scores, tagset, score='f1-score'):
    fig, ax = plt.subplots(figsize=(12, 5))
    barWidth = 0.2
    bars = [[data[tag] for tag in tagset if tag != 'O'] for data in scores.values()]
    rnge = np.arange(0, (len(bars[0])*1.2), 1.2)
    r = [rnge]
    for i in range(1, len(bars)):
        r.append([x + barWidth for x in r[i-1]])
       

    
    colors = ['#0072B2', '#D55E00', '#CC79A7', '#E69F00', '#56B4E9'] # Colorblind-friendly palette
    for i in range(len(bars)):
        ax.bar(r[i], bars[i], color=colors[i], width=barWidth, label=f'{list(scores.keys())[i]}')
    ax.set_xticks([r + barWidth for r in rnge])
    ax.set_xticklabels(tagset, rotation=30)
    ax.tick_params(bottom=False, left=False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_color('#DDDDDD')
    ax.set_axisbelow(True)
    ax.yaxis.grid(True, color='#EEEEEE')
    ax.xaxis.grid(False)
    ax.set_xlabel('Token classes', labelpad=15, color='#333333')
    ax.set_ylabel(score, labelpad=15, color='#333333')
    plt.legend(loc=(1.04, 0.7))
    plt.show()

    return fig

In [32]:
models = ['gpt4o', 'gpt4.1-mini']#['gpt3.5', 'gpt4', 'gpt4o', 'o1-mini', 'gpt4.1-mini']
metrics = ['precision', 'recall', 'f1-score']

In [19]:
path = join('predictions','token_classification_' + models[0])

tagset = sorted({label for sentence in test_set['tags'] for tag in sentence for label in (tag if isinstance(tag, list) else [tag])})

predictions = load_predictions(path)
scores = evaluate_multilabel(predictions[0], test_set, tagset)
for tag, score in scores.items():
    print(f"{tag}: Precision={score['precision']:.2f}, Recall={score['recall']:.2f}, F1={score['f1-score']:.2f}")

Domain-mark: Precision=1.00, Recall=0.41, F1=0.58
ENE-Misc: Precision=0.03, Recall=0.01, F1=0.02
ENE-Person: Precision=0.45, Recall=0.03, F1=0.05
ENE-Spatial: Precision=0.59, Recall=0.19, F1=0.28
Head: Precision=0.92, Recall=0.52, F1=0.66
Latlong: Precision=0.93, Recall=0.41, F1=0.57
NC-Person: Precision=0.82, Recall=0.14, F1=0.24
NC-Spatial: Precision=0.80, Recall=0.34, F1=0.48
NP-Misc: Precision=0.19, Recall=0.11, F1=0.14
NP-Person: Precision=0.75, Recall=0.41, F1=0.53
NP-Spatial: Precision=0.93, Recall=0.50, F1=0.65
O: Precision=0.79, Recall=0.98, F1=0.88
Relation: Precision=0.62, Recall=0.36, F1=0.46
macro avg: Precision=0.68, Recall=0.34, F1=0.43
micro avg: Precision=0.79, Recall=0.76, F1=0.78


/opt/homebrew/Caskroom/miniforge/base/envs/ner-llm-py311/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['NC-Misc', 'NE-Misc'] will be ignored
  warnings.warn(


In [34]:
for model in models:
    print(model)
    path = join('predictions','token_classification_' + model)
    tagset = sorted({label for sentence in test_set['tags'] for tag in sentence for label in (tag if isinstance(tag, list) else [tag])})
    predictions = load_predictions(path)
    classification_reports = evaluate(predictions, test_set, tagset)

    for metric in metrics:
        scores = get_avg_scores(classification_reports, metric)
        print(f'{metric}', end=': ')
        print('micro:', scores['micro avg'], end='\t')
        print('macro:', scores['macro avg'], end='\t')
        #print("{:.2f}".format(scores['micro avg']), end='\t')
    print()

gpt4o
precision: micro: 0.7907453196750265	macro: 0.6789836136611052	recall: micro: 0.7631417467784823	macro: 0.3389642880678281	f1-score: micro: 0.7766983554229409	macro: 0.4259636986720865	
gpt4.1-mini
precision: micro: 0.7848625490896108	macro: 0.6223895852016401	recall: micro: 0.7494375127838003	macro: 0.5321373683818218	f1-score: micro: 0.7667410714285714	macro: 0.5208862026087302	


/opt/homebrew/Caskroom/miniforge/base/envs/ner-llm-py311/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['NC-Misc', 'NE-Misc'] will be ignored
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ner-llm-py311/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['NC-Misc'] will be ignored
  warnings.warn(


In [ ]:
eval_scores = {}

for metric in metrics:
    for model in models:
        path = join('predictions','token_classification_' + model)
        tagset = sorted({label for sentence in test_set['tags'] for tag in sentence for label in (tag if isinstance(tag, list) else [tag])})
    
        predictions = load_predictions(path)
        classification_reports = evaluate(predictions, test_set, tagset)
        eval_scores[model] = get_avg_scores(classification_reports, metric)

    bar_plot(eval_scores, tagset, metric)
